In [1]:
import numpy as np
import pandas as pd
import gensim
import konlpy

from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import InputLayer, Dense
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Concatenate, Reshape, Flatten
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.utils import model_to_dot
from IPython.display import SVG

In [2]:
data_path = '../data/Chatbot_data-master/ChatbotData.csv'
df = pd.read_csv(data_path)
df

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [3]:
df = df.dropna()
labels = df['label'].unique()
print(labels)
df

[0 1 2]


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [4]:
# DataFrame을 레이블로 그룹화
grouped_obj = df.groupby('label')

train_x = []
train_y = []

test_x = []
test_y = []

# 레이블의 개수만큼 반복
# labels=[0, 1, 2]이므로 총 3번 반복
for label in labels:
    print(label)
    # 그룹화된 DataFrame에서 label이 동일한 그룹만 추출(?)
    # 반환값은 해당 데이터들의 인덱스
    label_group = grouped_obj.groups[label]
    print(label_group)
    
    # 반환값이 인덱스이므로, 전체 데이터에 .iloc()을 사용해
    # 해당 row들만 추출하고, .values로 데이터만 추출
    sentences = df.iloc[label_group]['Q'].values
    print(sentences)
    cur_labels = df.iloc[label_group]['label'].values
    
    # 7:3으로 학습 데이터와 평가 데이터 나눠서 저장하기.
    split_index = int(len(sentences)*0.7)
    train_x.extend(sentences[:split_index])
    train_y.extend(cur_labels[:split_index])
    
    test_x.extend(sentences[split_index:])
    test_y.extend(cur_labels[split_index:])

0
Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            5280, 5281, 5282, 5283, 5284, 5285, 5286, 5287, 5288, 5289],
           dtype='int64', length=5290)
['12시 땡!' '1지망 학교 떨어졌어' '3박4일 놀러가고 싶다' ... '힘든 시기가 지나갔으면' '힘든 데도 날 도와준 사람'
 '힘차게 시작']
1
Int64Index([5290, 5291, 5292, 5293, 5294, 5295, 5296, 5297, 5298, 5299,
            ...
            8850, 8851, 8852, 8853, 8854, 8855, 8856, 8857, 8858, 8859],
           dtype='int64', length=3570)
['1000일 만난 여자친구와 이별' '10년 연애. 헤어졌습니다.' '10년 연애사 되돌아보니 다 부질없네' ... '힘이 드네'
 '힘이 없어' '힘이드네.여자들이란.']
2
Int64Index([ 8860,  8861,  8862,  8863,  8864,  8865,  8866,  8867,  8868,
             8869,
            ...
            11813, 11814, 11815, 11816, 11817, 11818, 11819, 11820, 11821,
            11822],
           dtype='int64', length=2963)
['짝사랑만큼 고통스러운 건 없겠지.' '1년 넘게 만났는데 지금도 불타올라' '1년 동거 중인데 계속 좋아' ...
 '흑기사 해주는 짝남.' '힘든 연애 좋은 연애라는게 무슨 차이일까?' '힘들어서 결혼할까봐']


In [5]:
print(len(train_x))
print(len(train_y))
print(len(test_x))
print(len(test_y))
print(len(test_x)+len(train_x))

8275
8275
3548
3548
11823


In [6]:
# 문장의 최대길이 구하는 함수 작성
def calc_max_len(sents):
    # max()함수로 최대값을 구하며,
    # [len(sent) for sent in sents]는
    # 전체 문장의 하나씩 가져와 {for sent in sents}
    # 각 문장의 길이를 리스트에 저장 [len(sent) ]
    return max([len(sent) for sent in sents])

# 리스트 더하기를 사용해서 전체 문장 사용 즉,
# train_x랑 test_x를 더해 7:3을 10으로 변경
max_len = calc_max_len(train_x+test_x)
print(max_len)

# w2v pre-trained 모델의 embedding_size
embedding_size = 200

56


In [7]:
# 문장 전처리를 위한 함수 작성
def sent_prep(sent, tokenizer, wv_model):
    sent_vec = []
    
    # 전처리 진행
    words = [word for word, _ in tokenizer.pos(sent)]
    
    # 전처리 진행 후 남은 단어들을
    # w2v pre-trained 모델을 사용해 벡터로 변경
    for word in words:
        try:
            sent_vec.append(wv_model.wv[word])
        # pre-trained 모델에 단어가 없으면 넘어가도록 진행
        except KeyError:
            continue
    
    # 진행 후 sent_vec이 비어있으면, None을 반환
    if not sent_vec:
        return None
    
    # 아니면, zero-padding을 진행
    else: 
        padding_size = max_len - len(sent_vec)
        for _ in range(padding_size):
            sent_vec.insert(0, np.zeros(embedding_size))
        return np.asarray(sent_vec).astype('float')

In [ ]:
# tokenizer, embedding_model 생성
okt = konlpy.tag.Okt()
wv = gensim.models.Word2Vec.load('../data/ko.bin')

# 처음인지 아닌지를 위한 flag 설정
init_flag = True

# 학습 데이터의 입력과 출력을 zip으로 함께 받아오면서 반복
for sentence, label in zip(train_x, train_y):
    # 전처리 함수로 입력(문장) 전처리
    sent_vec = sent_prep(sentence, okt, wv)
    
    # 함수의 반환값이 None이 아닐때
    if sent_vec is not None:
        # 반복의 첫번째라면
        if init_flag:
            # x_train과 y_train에 각각의 데이터 할당
            # CNN 모델을 위해 shape 변경
            x_train = sent_vec.reshape(1, sent_vec.shape[0], sent_vec.shape[1])
            y_train = to_categorical(label, num_classes=len(labels)).reshape(1, -1)
            # 첫번째 반복이 끝났다는 표시
            init_flag = False
        # 두번째 반복부터
        else:
            # 기존의 x_train과 y_train에 새로운 데이터를
            # concatenate()를 사용해 추가
            x_train = np.concatenate([x_train, sent_vec.reshape(1, sent_vec.shape[0], sent_vec.shape[1])], axis=0)
            y_train = np.concatenate([y_train, to_categorical(label, num_classes=len(labels)).reshape(1, -1)], axis=0)

In [ ]:
# 직전 셀과 동일하게
# 평가 데이터에도 진행
init_flag = True
for sentence, label in zip(test_x, test_y):
    sent_vec = sent_prep(sentence, okt, wv)
    if sent_vec is not None:
        if init_flag:
            x_test = sent_vec.reshape(1, sent_vec.shape[0], sent_vec.shape[1])
            y_test = to_categorical(label, num_classes=len(labels)).reshape(1, -1)
            init_flag = False
        else:
            x_test = np.concatenate([x_test, sent_vec.reshape(1, sent_vec.shape[0], sent_vec.shape[1])], axis=0)
            y_test = np.concatenate([y_test, to_categorical(label, num_classes=len(labels)).reshape(1, -1)], axis=0)

In [ ]:
# 학습 데이터 CNN을 위한 형태로 shape 변경
# 및 데이터 확인
x_train = x_train.reshape(x_train.shape[0], 1, x_train.shape[1], x_train.shape[2])
print(len(x_train))
print(x_train.shape)

print(len(y_train))
print(y_train.shape)

In [ ]:
# 평가 데이터 CNN을 위한 형태로 shape 변경
# 및 데이터 확인
x_test = x_test.reshape(x_test.shape[0], 1, x_test.shape[1], x_test.shape[2])
print(len(x_test))
print(x_test.shape)
print(len(y_test))
print(y_test.shape)

In [ ]:
# 자연어 처리용 CNN 레이어 생성 및 연결
input_layer = Input(shape=(1, max_len, embedding_size))
conv_1_layer = Conv2D(filters=64, kernel_size=(2, embedding_size), padding='same', strides=1)
conv_2_layer = Conv2D(filters=64, kernel_size=(3, embedding_size), padding='same', strides=1)
conv_3_layer = Conv2D(filters=64, kernel_size=(4, embedding_size), padding='same', strides=1)
max_pool_1_layer = MaxPool2D(64, padding='same')
max_pool_2_layer = MaxPool2D(64, padding='same')
max_pool_3_layer = MaxPool2D(64, padding='same')
concat_layer = Concatenate()
dense_layer = Dense(64, activation='relu')
flatten_layer = Flatten()
output_layer = Dense(3, activation='softmax')

x_1 = max_pool_1_layer(conv_1_layer(input_layer))
x_2 = max_pool_2_layer(conv_2_layer(input_layer))
x_3 = max_pool_3_layer(conv_3_layer(input_layer))

concat_out = concat_layer([x_1, x_2, x_3])
dense_out = dense_layer(concat_out)
output = output_layer(flatten_layer(dense_out))

In [ ]:
# 모델 생성
model = Model(input_layer, output)

In [ ]:
# summary()로 모델 확인
model.summary()

In [ ]:
# SVG로 모델 시각화
SVG(model_to_dot(model, show_shapes=True, dpi=55).create(prog='dot', format='svg'))

In [ ]:
# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# 모델 학습
model.fit(x_train, y_train, epochs=10, batch_size=258)

In [ ]:
# 정확도만 평가할 수 있는 함수 작성
def _eval(sentences, labels):
    correct_cnt = 0
    
    # 문장이랑 레이블 받아서 진행할게요.
    for sentence, label in zip(sentences, labels):
        # 모델 입력 데이터 array로 바꾸면서, 타입도 변경할게요.
        x = np.asarray(sentence).astype('float')
        
        # CNN 계열은 입력이 4차원이기 때문에 shape 변경할게요.
        x = x.reshape(x.shape[0], 1, x.shape[1], embedding_size)
        
        # label은 (0, 1)로 되어있으니까, max값의 인덱스로
        # label 설정할게요.
        y = list(label)
        y = y.index(max(y))
        
        # 모델로 x 분류 진행할게요.
        predict = np.squeeze(model.predict(x))
        predict = predict.argmax()
        if predict == y:
            correct_cnt += 1
    return correct_cnt

In [ ]:
# 평가 데이터로 정확도 평가 진행
ccnt = _eval(x_test, y_test)
print(ccnt, ' / ', len(x_test), ' : ' , ccnt/len(x_test))